In [43]:
import abc
from abc import abstractmethod

class Parent(abc.ABC):
    def __init__(self):
        self.params = {}
    @abstractmethod
    def get_param(self,value):
        if value in self.params.keys():
            return self.params.get(value)
        else:
            raise ValueError("No such value in params:")
    @abstractmethod
    def set_param(self,value,data):
        self.params[value] = data
        return self.params

# When you gonna try to do such...
# parent = Parent()

# It will raise TypeError: Can't instantiate abstract  
# class Parent without an implementation for abstract methods 
# 'get_param', 'set_param'



# Only after defining methods in a child class you will get success
class Child(Parent):
    def __init__(self):
        self.params = {}
    def get_param(self,value):
        if value in self.params.keys():
            return self.params.get(value)
        else:
            raise ValueError("No such value in params:")
    def set_param(self,value,data):
        self.params[value] = data
        return self.params


child = Child()

child.set_param(value="value one",data="value ones data")
child.set_param(value="value two",data="value two data")
child.set_param(value="value tree",data="value three data")

child.__dict__

child.get_param("value x")
# child.__dict__

ValueError: No such value in params:

In [49]:
! pip install paramiko

  Using cached bcrypt-4.2.0-cp39-abi3-macosx_10_12_universal2.whl.metadata (9.6 kB)
Using cached bcrypt-4.2.0-cp39-abi3-macosx_10_12_universal2.whl (472 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 315.2 kB/s eta 0:00:00 kB/s eta 0:00:04:06


In [58]:
# lets create the class SSH Context manager

import paramiko
import time
import abc
from abc import abstractmethod


class BaseSSH(abc.ABC):
    def __init__(self, ip, username, password):
        self.ip = ip
        self.username = username
        self.password = password
        self._MAX_READ = 10000

        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        client.connect(
            hostname=ip,
            username=username,
            password=password,
            look_for_keys=False,
            allow_agent=False)

        self._ssh = client.invoke_shell()
        time.sleep(1)
        self._ssh.recv(self._MAX_READ)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._ssh.close()

    def close(self):
        self._ssh.close()

    @abc.abstractmethod
    def send_command(self, command):
        """Send command and get command output"""

    @abc.abstractmethod
    def send_config_commands(self, commands):
        """Send configuration command(s)"""




# So child class are here

class CiscoSSH(BaseSSH):
    device_type = 'cisco_ios'
    def __init__(self, ip, username, password, enable_password,
                 disable_paging=True):
        super().__init__(ip, username, password)
        self._ssh.send('enable\n')
        self._ssh.send(enable_password + '\n')
        if disable_paging:
            self._ssh.send('terminal length 0\n')
        time.sleep(1)
        self._ssh.recv(self._MAX_READ)

    def send_command(self, command):
        self._ssh.send(command + '\n')
        time.sleep(0.5)
        result = self._ssh.recv(self._MAX_READ).decode('ascii')
        return result

    def config_mode(self):
        self._ssh.send('conf t\n')
        time.sleep(0.5)
        result = self._ssh.recv(self._MAX_READ).decode('ascii')
        return result

    def exit_config_mode(self):
        self._ssh.send('end\n')
        time.sleep(0.5)
        result = self._ssh.recv(self._MAX_READ).decode('ascii')
        return result

    def send_config_commands(self, commands):
        result = self.config_mode()
        result += super().send_config_commands(commands)
        result += self.exit_config_mode()
        return result
